# <b> Notebook Basics Data Scientists Package Import

In [4]:
%run cp_00_00_import_and_functions.ipynb

In [5]:
import shap

# Explicabilité

### custom_explain()

In [ ]:
def custom_explain(dataframe, drop_columns, model, color, labels):
    '''
    Compute the shapely values for explainability and exxport them into images

    Parameters
    ----------
    X : pd.DataFrame
        Features dataframe as used by the model for predction
    model : model
        Model to explain
    color : array
        Array of hex code for the classes
    labels : string
        Array of labels of the target class (must be in the same order as the colors in the array)
    
    Returns
    -------
    explainer : explainer
        Explainer model
    shap_values : array
        Shap values computed for explaination
    '''
    
    # Shap
    X = dataframe.drop(columns = drop_columns)
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X,approximate=True)
    if type(shap_values)==list:
            fig = plt.figure(figsize=(20,10))
            fig.set_size_inches(5,8)
            
            # Creation de la colormap
            class_inds = np.argsort([-np.abs(shap_values[i]).mean() for i in range(len(shap_values))])
            cmap = []
            for val in class_inds:
                cmap.append(color[val])
            cmap = plt_colors.ListedColormap(cmap)
            
            shap.summary_plot(shap_values, X, plot_type="bar", show=False, class_names= labels, color = cmap)
            #plt.tight_layout()
            plt.savefig('%s/output/multiclass.png'%(path_notebooks),dpi=300, bbox_inches='tight')
            for i in range(len(shap_values)):
                plt.figure(figsize=(20,10))
                shap.summary_plot(shap_values[i], X, show=False)
                #plt.tight_layout()
                plt.savefig('%s/output/Class%i.png'%(path_notebooks,i),dpi=300, bbox_inches='tight')
    else:
        plt.figure(figsize=(20,10))
        shap.summary_plot(shap_values,X,show=False)
        #plt.tight_layout()
        plt.savefig('%s/output/result.png'%(path_notebooks),dpi=300, bbox_inches='tight')
        
    return explainer, shap_values

### custom_feature_importance()

In [ ]:
def custom_feature_importance(features_name, shap_values):
    '''
    Create the feature importance DataFrame
    WARNING: Maybe the function needs correction for vals variable computation with the .mean() beacause shap values will return an array not a matrix

    Parameters
    ----------
    features_name : array
        List of all features name (Ex: X.columns.tolist())
    shap_values : array
        Shap values computed by the custom_explain() function
    
    Returns
    -------
    feature_importance : pd.DataFrame
        Return a DataFrame with all features names and its importance
    '''
    vals = np.abs(shap_values).mean(1).mean(0)
    feature_importance = pd.DataFrame(list(zip(features_name, vals)), columns=['features', 'importance'])
    feature_importance.sort_values(by=['importance'], ascending=False, inplace=True)
    return feature_importance